In [1]:
import numpy as np
import cv2 as cv
import glob
import os
from google.colab.patches import cv2_imshow

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Función para cargar y aumentar imágenes
def augment_images(image_paths, scale_factor=1.2, rotation_angle=10):
    augmented_images = []
    for path in image_paths:
        image = cv.imread(path)
        augmented_images.append(image)

        # Escala
        scaled_image = cv.resize(image, None, fx=scale_factor, fy=scale_factor, interpolation=cv.INTER_LINEAR)
        augmented_images.append(scaled_image)

        # Rotación
        rows, cols, _ = image.shape
        rotation_matrix = cv.getRotationMatrix2D((cols/2, rows/2), rotation_angle, 1)
        rotated_image = cv.warpAffine(image, rotation_matrix, (cols, rows))
        augmented_images.append(rotated_image)

    return augmented_images

In [4]:
# Función para mostrar imágenes
def show_images(images):
    for idx, img in enumerate(images):
        cv2_imshow(img)
        cv.waitKey(500)

# criterios de terminación
criterios = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# preparar puntos de objeto, como (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)

# Matrices para almacenar puntos de objetos y puntos de imágenes de todas las imágenes.
objpoints = [] # punto 3d en el espacio del mundo real
imgpoints = [] # puntos 2d en el plano de la imagen.

image_paths = glob.glob("/content/drive/MyDrive/Visión por computador/Seguimientos de clase/SEM_11/left/*.jpeg")


ruta_carpeta = '/content/drive/MyDrive/Visión por computador/Seguimientos de clase/SEM_11/left'
imagenes = glob.glob(os.path.join(ruta_carpeta, '*.jpeg'))

# Verificar si hay al menos una imagen en la lista
if len(imagenes) > 0:
    # Aumentar las imágenes
    all_images = augment_images(imagenes)

    # Mostrar imágenes originales y aumentadas
    show_images(all_images)

    for img_path in imagenes:
        img = cv.imread(img_path)
        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

        # Encuentra las esquinas del tablero de ajedrez.
        ret, corners = cv.findChessboardCorners(gray, (7,6), None)

        # Si lo encuentra, agregue puntos de objeto, puntos de imagen (después de refinarlos)
        if ret == True:
            objpoints.append(objp)

            corners2 = cv.cornerSubPix(gray, corners, (11,11), (-1,-1), criterios)
            imgpoints.append(corners2)

            # Dibuja y muestra las esquinas.
            img_drawn = img.copy()
            cv.drawChessboardCorners(img_drawn, (7,6), corners2, ret)
            cv2_imshow(img_drawn)
            cv.waitKey(500)

    # Calibración de la cámara
    ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

    if ret:
        print("La calibración de la cámara fue exitosa.")
    else:
        print("La calibración de la cámara falló.")
else:
    print("No se encontraron imágenes en la carpeta especificada.")


Output hidden; open in https://colab.research.google.com to view.

In [5]:
# Emparejar puntos de características entre dos imágenes
def match_features(image1, image2):
    # Convertir imágenes a escala de grises
    gray1 = cv.cvtColor(image1, cv.COLOR_BGR2GRAY)
    gray2 = cv.cvtColor(image2, cv.COLOR_BGR2GRAY)

    # Inicializar el detector de características y el descriptor
    orb = cv.ORB_create()

    # Encontrar puntos clave y descriptores en las imágenes
    kp1, des1 = orb.detectAndCompute(gray1, None)
    kp2, des2 = orb.detectAndCompute(gray2, None)

    # Inicializar el matcher de fuerza bruta
    bf = cv.BFMatcher(cv.NORM_HAMMING, crossCheck=True)

    # Hacer coincidir descriptores entre las dos imágenes
    matches = bf.match(des1, des2)

    # Extraer coordenadas de puntos coincidentes
    pts1 = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    pts2 = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

    return pts1, pts2

In [6]:
# Calcular la matriz esencial entre dos imágenes
def calculate_essential_matrix(pts1, pts2, mtx):
    # Normalizar los puntos de imagen
    pts1_norm = cv.undistortPoints(pts1, mtx, None)
    pts2_norm = cv.undistortPoints(pts2, mtx, None)

    # Calcular la matriz esencial
    F, mask = cv.findFundamentalMat(pts1_norm, pts2_norm, cv.FM_LMEDS)
    E = mtx.T @ F @ mtx

    return E

In [7]:
# Calcular la matriz de proyección de la segunda cámara
def calculate_projection_matrix(E, pts1, pts2):
    _, R, t, _ = cv.recoverPose(E, pts1, pts2)
    P2 = np.hstack((R, t))

    return P2

In [8]:
# Triangulación para calcular las coordenadas 3D
def triangulate_points(pts1, pts2, P1, P2):
    # Triangulación de puntos
    points_4d = cv.triangulatePoints(P1, P2, pts1, pts2)

    # Convertir coordenadas homogéneas a coordenadas cartesianas
    points_3d = cv.convertPointsFromHomogeneous(points_4d.T)

    return points_3d.reshape(-1, 3)

In [9]:
# Calcular la matriz de proyección de la segunda cámara
def calculate_projection_matrix(E, pts1, pts2):
    _, R, t, _ = cv.recoverPose(E, pts1, pts2)
    R1 = np.eye(3)
    t1 = np.zeros((3, 1))
    P1 = np.hstack((R1, t1))
    P2 = np.hstack((R, t.reshape(-1, 1)))
    return P1, P2

# Reconstrucción 3D a partir de varias imágenes
def reconstruct_3d_multiple_images(imgpaths, mtx, dist):
    # Inicializar la lista de puntos 3D reconstruidos
    reconstructed_points = []

    # Iterar sobre pares de imágenes adyacentes
    for i in range(len(imgpaths) - 1):
        # Cargar imágenes
        img1 = cv.imread(imgpaths[i])
        img2 = cv.imread(imgpaths[i+1])

        # Encontrar puntos de características coincidentes
        pts1, pts2 = match_features(img1, img2)

        # Calcular la matriz esencial
        E = calculate_essential_matrix(pts1, pts2, mtx)

        # Calcular las matrices de proyección de ambas cámaras
        P1, P2 = calculate_projection_matrix(E, pts1, pts2)

        # Triangulación de puntos
        points_3d = triangulate_points(pts1, pts2, P1, P2)

        # Agregar puntos 3D reconstruidos a la lista
        reconstructed_points.append(points_3d)

    return reconstructed_points

# Reconstrucción 3D
reconstructed_points = reconstruct_3d_multiple_images(imagenes, mtx, dist)


In [10]:
reconstructed_points

[array([[ 9.85973179e-01, -1.10897601e-01,  2.10992433e-03],
        [ 1.01536012e+00,  4.50669229e-03,  3.18550132e-03],
        [ 1.03680348e+00,  8.97447243e-02,  2.37227441e-03],
        [ 1.07388842e+00,  2.24108338e-01,  3.77797312e-03],
        [ 1.06085622e+00,  1.79425433e-01,  2.11470365e-03],
        [ 9.97841179e-01, -5.87559231e-02,  2.01779930e-03],
        [ 1.81363809e+00,  3.11338139e+00,  7.26823509e-03],
        [ 1.02688766e+00,  4.61949855e-02,  2.89534754e-03],
        [ 1.01551664e+00,  4.47270414e-03,  2.89646070e-03],
        [ 1.03590763e+00,  8.59729946e-02,  3.37359309e-03],
        [ 3.28105807e+00,  8.84521103e+00,  2.06173863e-02],
        [ 1.09135103e+00,  3.04855675e-01,  2.36743363e-03],
        [ 1.03586102e+00,  8.05375949e-02,  2.82591768e-03],
        [ 1.06741917e+00,  2.07587585e-01,  3.04791424e-03],
        [ 1.06290281e+00,  1.92597985e-01,  2.20278907e-03],
        [ 1.02038455e+00,  2.34540291e-02,  2.11695628e-03],
        [ 1.02427042e+00